# Low Inherent Bellman Errorと強化学習

参考
* [Learning Near Optimal Policies with Low Inherent Bellman Error](https://arxiv.org/abs/2003.00153)

テーブル形式MDPでは様々な良い強化学習の結果が得られていますが，実用上はなんらかの関数近似を入れる必要が出てきます．
一番便利なのは線形関数近似ですが，最適価値が線形に表されるとしても，Fitted Q iterationは発散することがあります（[RL_General_fitted_Q_iteration.ipynb](RL_General_fitted_Q_iteration.ipynb)でやりましたね）．

どのようなケースならば収束や有限サンプルでの性能が保証されるでしょうか？
すべての$Q^\pi$が線形に表現できる場合については結果が出ています（[Least-Squares Policy Iteration](https://www.jmlr.org/papers/v4/lagoudakis03a.html)など）．
この線形に近似できる条件を**LSPI条件**，と呼ぶことにします．

一方で，近似誤差が$\left\|\widehat{Q}^\pi-Q^\pi\right\| \leqslant \epsilon$であり，$\epsilon \gtrsim 1 / \sqrt{d}$のとき，サンプル効率は$d$について指数的になることが知られています（[Is a Good Representation Sufficient for Sample Efficient Reinforcement Learning?](https://arxiv.org/abs/1910.03016)）．

上で紹介したものは方策反復ベースのアルゴリズムについてですが，価値反復についても結果が得られています．
それを紹介するために，inherent Bellman errorを導入しましょう．

---

準備

* ベルマン最適作用素：$\mathcal{T}_t\left(Q_{t+1}\right)(s, a)=r_t(s, a)+\mathbb{E}_{s^{\prime} \sim p_t(s, a)} \max _{a^{\prime}} Q_{t+1}\left(s^{\prime}, a^{\prime}\right)$
* ベクトルの集合：$\mathcal{B}_t \stackrel{\text { def }}{=}\left\{\theta_t \in \mathbb{R}^{d_t}|| \phi_t(s, a)^{\top} \theta_t \mid \leqslant D, \forall(s, a)\right\}$
* 価値関数：$Q_t\left(\theta_t\right)(s, a)=\phi_t(s, a)^{\top} \theta_t, \quad V_t\left(\theta_t\right)=\max _a \phi_t(s, a)^{\top} \theta_t$
* $\mathcal{Q}_t \stackrel{\text { def }}{=}\left\{Q_t\left(\theta_t\right) \mid \theta_t \in \mathcal{B}_t\right\}, \mathcal{V}_t \stackrel{\text { def }}{=}\left\{V_t\left(\theta_t\right) \mid \theta_t \in \mathcal{B}_t\right\}$

このとき，（線形な特徴ベクトル$\phi$についての）Inherent Bellman errorは次で定義されます．

次を最大にする$t \in [H]$における値であり，$\mathcal{I}$で表記されます．

$$
\sup _{\theta_{t+1} \in \mathcal{B}_{t+1}} \inf _{\theta_t \in \mathcal{B}_t} \sup _{(s, a) \in \mathcal{S} \times \mathcal{A}} \left| \phi_t(s, a)^{\top} \theta_t \left(\mathcal{T}_t Q_{t+1}\left(\theta_{t+1}\right)\right)(s, a) \right|.
$$

ちなみにボーナスの追加などを考慮して，クリップした値について考える場合もあります（[Optimism in Reinforcement Learning with Generalized Linear Function Approximation](https://arxiv.org/abs/1912.04136)など．）
クリップしたほうが強い仮定になります．

$\mathcal{I}=0$の条件をLSVI条件と呼ぶことにします（多分Linear Bellman Completenessとも呼ばれる？）．
LSVI条件が成立しているとき，次のように報酬が線形になり，また，次状態の価値も線形に表現することができます．

$$
\inf _{\theta_t^R \in \mathcal{B}_t} \sup _{(s, a) \in \mathcal{S} \times \mathcal{A}}\left|r_t(s, a)-\phi_t(s, a)^{\top} \theta_t^R\right|=0
$$

$$
\sup _{\theta_{t+1} \in \mathcal{B}_{t+1}} \inf _{\theta_t^P \in \mathcal{B}_t} \sup _{(s, a) \in \mathcal{S} \times \mathcal{A}} | \mathbb{E}_{s^{\prime} \sim p_t(s, a)} V_{t+1}\left(\theta_{t+1}\right)\left(s^{\prime}\right) 
-\phi_t(s, a)^{\top} \theta_t^P | =0 .
$$

---

**LSPIとLSVI条件の違い**

* [Linear MDP](RL_General_linearMDP.ipynb.ipynb)は$\mathcal{I}=0$満たしますが，$\mathcal{I}=0$はlinear MDPとは限りません．その意味で，Linear MDPを一般化しています．
* また，同様の意味で，LSPI条件はLinear MDPを一般化しています．
* LSVI条件が満たされているとき，Bellman rankは特徴ベクトルの次元と一致します．つまり，Bellman rankはLSVI条件を一般化しています．
* LSVI条件とLSPI条件は別物です．実際，次が成立します．

$\mathcal{I}=0$だが，
$$
\exists \pi, \exists t \in[H], \nexists \theta_t^\pi \in \mathbb{R}^{d_t} \quad \text { s.t. } \quad Q_t^\pi=\phi_t(s, a)^{\top} \theta_t .
$$
のMDPが存在する．また，
$$
\forall \pi, \forall t \in[H], \exists \theta_t^\pi \quad \text { that satisfies } \quad Q_t^\pi(s, a)=\phi_t(s, a)^{\top} \theta_t^\pi
$$
だが，$\mathcal{I} > 0$のMDPが存在する．

---

このLSVI条件について，
* G-optimal design で学習するアプローチ：[LSVIアルゴリズム](https://rltheorybook.github.io/)
* オンラインの設定：[ELEANORアルゴリズム](https://arxiv.org/abs/2003.00153)

などが存在しています．ELEANORアルゴリズムはMinimax最適です．
